# Challenge 2
An important aspect of pragmatic vector space methods is the ability to handle vectors and matrices.
A large collection of linear algebra functions is available in [SciPy.linalg](https://docs.scipy.org/doc/scipy/reference/linalg.html).
These functions can be employed in conjunction with the tools available in [NumPy](http://www.numpy.org/).
We note that the main object in NumPy is the homogeneous multidimensional array.

## Matrix
We begin by creating a simple matrix.
One possible approach to complete this task is to use ```scipy.linalg.circulant(c)```.

In [20]:
from scipy.linalg import circulant
my_circ_matrix = circulant([1, 2, 3])

Alternatively, you can construct the familiar discrete Fourier transform matrix with ```scipy.linalg.dft(n)```.

In [21]:
from scipy.linalg import dft
my_dft_matrix = dft(3)

The inverse of a matrix can be computed using ```scipy.linalg.inv(a)```.

In [22]:
from scipy.linalg import inv
my_idft_matrix = inv(my_dft_matrix)

The operation ```numpy.dot(a, b)``` computes the dot product of two arrays.
For 2-D arrays it is equivalent to matrix multiplication, and for 1-D arrays to inner product of vectors (without complex conjugation).

In [23]:
import numpy as np
matrix_prod1 = np.dot(my_dft_matrix, my_circ_matrix)
matrix_prod2 = np.dot(matrix_prod1, my_idft_matrix)

np.set_printoptions(suppress=True)
print(matrix_prod2)

[[ 6.0-0.j         0.0+0.j        -0.0+0.j       ]
 [-0.0-0.j        -1.5+0.8660254j -0.0-0.j       ]
 [ 0.0-0.j         0.0-0.j        -1.5-0.8660254j]]


### Questions
These steps and their solutions immediately bring up three questions.
 * Are circulant matrices always diagonalized by the discrete Fourier transform matrix and its inverse?
 * Are product of circulant matrices (of a same size) always circulant matrices?
 * Do all pairs of circulant matrices commute under matrix multiplication?
 
### Solutions
 * Yes. Since the DFT matrix is a unitray matrix ($FF^H=F^HF=I$), so $F^{-1}=F^H$. According to the definition, all circulant matrices are made diagonal by the DFT, regardless of the generating vector $x$. $F^HXF=diag(\hat x)$ ($\hat x$ is the DFT of generating vector $x$), thus $F^{-1}XF=diag(\hat x)$.
 * Yes. Assume that $A,B$ are curculant matrices.
 \begin{align}
 AB&=F\cdot diag(\hat a)\cdot F^H\cdot F\cdot diag(\hat b)\cdot F^H\\
 &=F\cdot diag(\hat a)\cdot diag(\hat b)\cdot F^H\\
 &=F\cdot diag(\hat a\odot \hat b)\cdot F^H\\
 &=C(\mathcal F^{-1}(\hat a\odot \hat b))
 \end{align}
 $\odot$ is Hadamard product of vectors. Product of circulant matrices always circulant matrices whose generating vectors are the Hadamard product of the DFT of the old generating vectors.
 * Yes. According to the equation above, $BA=C(\mathcal F^{-1}(\hat b\odot \hat a))=C(\mathcal F^{-1}(\hat a\odot \hat b))=AB$.

## Determinant
The determinant of a square matrix is a value derived arithmetically from the coefficients of the matrix, and it summarizes a multivariable phenomenon with a signle number.
It can be computed with ```scipy.linalg.det(a)```.

In [24]:
from scipy.linalg import det
det(my_circ_matrix)

18.0

The code below demonstrates how to create a function in Python, how to vectorize a function so that it can be applied to the elements of a matrix, and how to use ```random```.

In [25]:
import math
from numpy import random

def my_log(x):
    return math.log(x)

my_vec_log = np.vectorize(my_log)

A_step1 = my_vec_log(my_circ_matrix) # Numpy already offers a vectorized natural logarithm.
# A_step1 = np.log(matrix_prod2)

max_index = 100000
my_identity = np.identity(len(A_step1))
current_value = 0.0
for my_index in range(0, max_index):
    permutation_matrix = random.permutation(my_identity)
    sign_permuation = det(permutation_matrix)
    current_value += sign_permuation*(np.exp(np.trace(np.dot(A_step1, permutation_matrix))))
a_step2 = math.factorial(len(A_step1)) * current_value / max_index
print(a_step2)

17.87166


#### Questions
It appears that the output of the loop above is close to the determinant of the circulant matrix ```my_circ_matrix```.
 * Go through the code and provide a compelling explain explanation of why these numbers are close.
 * Is this a property of circulant matrices, or would this finding extend to arbitrary matrices over the real numbers?
 
#### Solutions
 * 1. Take the natural logarithm of each element in the circulant matrix to get the corresponding logarithmic matrix.
2. Randomly generate a permutation matrix with the same order of the logarithmic matrix.
3. Multiply the logarithmic matrix with the permutation matrix to obtain a new logarithmic matrix.
4. Finding the trace of new logarithmic matrix, that is, finding the sum of the main diagonal elements. Take the exponential operation to the sum to obtain the product of the three elements in the different rows and columns in the circulant matrix. The sign of these products are determined by the determinant of the permutation matrix.
5. Repeat steps 2-4 for a number of times (100,000 in the code) and find the average of 100,000 products. The matrix in the code is 3-order, we know that the 3-order permutation matrix has a total of six, so we can get 6 kinds of products. When the number of random generation is large enough, we can approximately think that six products occur same times. So the average can be considered the average of six different products, and then multiply by 6 to get the sum of six different products. This result conforms to the definition of the determinant, so the result is very similar to the determinant.
 * Theoretically, this finding extends to arbitrary square matrices, as long as the number of random generation is large enough, the result is relatively reliable. Taking into account the actual situation, $n$-order matrix will have $n!$ different products, if $n$ is relatively large, then the number of random generation will be particularly large, the calculation will take a very long time.

### Tasks
 * Build code to explore the fact that the determinant function is multiplicative: $\mathrm{det}(AB) = \mathrm{det}(A) \mathrm{det}(B)$.

In [26]:
import numpy as np
from numpy import random
from scipy.linalg import det

n = random.randint(2,8)
matA = random.randint(1, 10, (n, n))
matB = random.randint(1, 10, (n, n))
x = np.round(det(np.dot(matA, matB)))
y = np.round(det(matA)*det(matB))
if(x == y):
    print("the determinant function is multiplicative: det(AB)=det(A)det(B)=",x)
else:
    print("det(AB)=", x, "det(A)det(B)=", y)

the determinant function is multiplicative: det(AB)=det(A)det(B)= 8391880.0
